In [ ]:
#Execute SQL and load AEMR.db database

%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [ ]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

In [ ]:
#Count the number of approved outaqges for each outage reason 

%%sql
SELECT
COUNT(*) AS Total_Number_Outages,
outage_reason,
year
FROM AEMR_Outage_Table
WHERE status = 'Approved'
Group By outage_reason, year
ORDER BY outage_reason;

#Forced outages have significantly increased in 2017 from the previous year at 1,264 in 2016 to 1,622 in 2017

In [ ]:
#Break the previous query down by month

%%sql
SELECT
year,
month,
COUNT(*) AS Total_Number_Outages,
outage_reason
FROM AEMR_Outage_table
WHERE status = 'Approved'
GROUP BY year, month;

#The amount of forced outages was down at the beginning of 2017 as compared to 2016, but spike from June through December.

In [ ]:
#Check the average duration (in days) and amount of the outage types by participant code and year

%%sql
SELECT
participant_code,
outage_reason,
year,
COUNT(*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS Average_Outage_Duration_In_Days
FROM aemr_outage_table
WHERE status = 'Approved'
GROUP BY participant_code, outage_reason, year
ORDER BY total_number_outage_events DESC, outage_reason, year;

#AURICON, GW and MELK have a higher number of outages and have higher average durations.

In [ ]:
#Classify participant risk by average duration
#High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)
#Medium Risk - On average, the participant is unavailable between 12 and 24 Hours
#Low Risk - On average, the participant is unavailable for less than 12 Hours

%%sql
WITH CTE1 AS (
SELECT
participant_code,
outage_reason,
year,
COUNT(*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS Average_Outage_Duration_In_Days
FROM aemr_outage_table
WHERE status = 'Approved'
GROUP BY participant_code, outage_reason, year
ORDER BY total_number_outage_events DESC, outage_reason, year
    )


SELECT
*,
CASE 
WHEN average_outage_duration_in_days > 1.00 THEN 'High Risk'
WHEN average_outage_duration_in_days BETWEEN 0.50 AND 1.00 THEN 'Medium Risk'
WHEN average_outage_duration_in_days BETWEEN 0.00 AND 0.50 THEN 'Low Risk'
ELSE 'N/A'
END AS Risk_Classification
FROM CTE1


In [ ]:
#Filter the number of outages and outage types
#High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20
#Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20
#Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10
#If Outage Type is not forced, then N/A

%%sql
WITH CTE1 AS (
SELECT
participant_code,
outage_reason,
year,
COUNT(*) AS Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))),2) AS Average_Outage_Duration_In_Days
FROM aemr_outage_table
WHERE status = 'Approved'
GROUP BY participant_code, outage_reason, year
ORDER BY total_number_outage_events DESC, outage_reason, year
    )

SELECT
*,
Total_Number_Outage_Events,
CASE 
WHEN average_outage_duration_in_days > 1.00 AND outage_reason = 'Forced' OR Total_Number_Outage_Events > 20 AND outage_reason = 'Forced' THEN 'High Risk'
WHEN average_outage_duration_in_days BETWEEN 0.50 AND 1.00 AND outage_reason = 'Forced' OR Total_Number_Outage_Events BETWEEN 10 AND 20 AND outage_reason = 'Forced' THEN 'Medium Risk'
WHEN average_outage_duration_in_days BETWEEN 0.00 AND 0.50 AND outage_reason = 'Forced' OR Total_Number_Outage_Events < 10 AND outage_reason = 'Forced' THEN 'Low Risk'
ELSE 'N/A'
END AS Risk_Classification,
outage_reason
FROM CTE1

In [ ]:
#Calculate proportion of forced outages over 2016-2017

%%sql
SELECT
COUNT(*) AS Total_Number_Outages,
SUM(CASE 
    WHEN outage_reason = 'Forced' THEN 1
    ELSE 0
    END) AS Total_Number_Forced_Outage_Events,
ROUND(SUM(CASE 
    WHEN outage_reason = 'Forced' THEN 1
    ELSE 0
    END) * 1.0 / COUNT(*) * 100, 2) AS Pct_Outage_Forced,
year
FROM aemr_outage_table
WHERE status = 'Approved'
GROUP BY year;

#Forced outages have significantly increased in 2017 over 2016.

In [1]:
#Calculate total outages, total duration, and total energy lost.
%%sql
SELECT
COUNT(*) AS Total_Number_Outages,
ROUND(SUM(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))), 2) AS Total_Duration_in_Days,
ROUND(SUM(energy_lost_mw), 2) AS Total_Energy_Lost,
outage_reason,
participant_code,
facility_code,
year
FROM aemr_outage_table
WHERE status = 'Approved'
GROUP BY outage_reason, participant_code, facility_code, year
ORDER BY total_energy_lost DESC, year DESC;

#AURICON, GW and MELK are consistently have the highest energy losses over 2016 and 2017.

UsageError: Cell magic `%%sql` not found.


In [ ]:
#Calculate average duration and average energy lost of forced outages

%%sql
SELECT
ROUND(AVG(ABS(JULIANDAY(end_time) - JULIANDAY(start_time))), 2) AS Avg_Duration_in_Days,
ROUND(AVG(energy_lost_mw), 2) AS Avg_Energy_Lost,
outage_reason,
participant_code,
facility_code,
year
FROM aemr_outage_table
WHERE status = 'Approved' AND outage_reason = 'Forced'
GROUP BY outage_reason, participant_code, facility_code, year
ORDER BY Avg_Energy_Lost DESC, year DESC;

In [ ]:
#Calculate average energy lost, total energy lost and percentage of energy lost due to forced outages.

%%sql
SELECT
ROUND(AVG(energy_lost_mw), 2) AS Avg_Energy_Lost,
ROUND(SUM(energy_lost_mw), 2) AS Total_Energy_Lost,
ROUND(SUM(energy_lost_mw) * 1.0 / (SELECT SUM(energy_lost_mw) 
                                  FROM aemr_outage_table 
                                  WHERE status = 'Approved' AND outage_reason = 'Forced')
                                  * 100, 2) AS Pct_Energy_Lost,
outage_reason,
participant_code,
facility_code,
year
FROM aemr_outage_table
WHERE status = 'Approved' AND outage_reason = 'Forced'
GROUP BY outage_reason, facility_code, participant_code, year
ORDER BY Total_Energy_Lost DESC;

#AURICON, GW and MELK have the highest percentages of energy losses.

In [ ]:
#Calculate total energy lost of AURICON, GW and MELK. Add in the description of outage of the highest total energy lost.
#Add in the the percentage of energy lost for each of those descriptions of outage.

%%sql
SELECT 
*
FROM (
SELECT
participant_code,
facility_code,
description_of_outage,
ROUND(SUM(energy_lost_mw), 2) AS Total_Energy_Lost,
ROUND(SUM(energy_lost_mw) / (SELECT
                      SUM(energy_lost_mw)
                      FROM aemr_outage_table
                      WHERE status = 'Approved' AND outage_reason = 'Forced') * 100, 2) AS Pct_Energy_Loss,
RANK() OVER (PARTITION BY participant_code, facility_code
            ORDER BY SUM(energy_lost_mw) DESC) AS rank
FROM aemr_outage_table
WHERE status = 'Approved' AND outage_reason = 'Forced'
AND participant_code IN ('AURICON', 'GW', 'MELK')
GROUP BY participant_code, facility_code, description_of_outage
ORDER BY participant_code, total_energy_lost DESC)
WHERE rank = 1;